In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# An example of a Low Liquidity Trade

In [19]:
import requests
import s3fs
import pandas as pd

from datetime import timedelta, datetime
import pendulum

import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from normalize.ticker_symbols import TsxToYhoo
from addFeatures.daily import DailyData 

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

import etl_moc

In [4]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [5]:
moc_file_lst = get_moc_file_lst.run("tsx-moc")

In [6]:
data_flpth = f"s3://{moc_file_lst[-1]}"
data_flpth

's3://tsx-moc/moc_tsx_20200407.csv'

In [7]:
moc_df = pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False)
moc_df["Symbol"].fillna("NA", inplace=True)

In [8]:
moc_df[moc_df["Symbol"].isna()==True]

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date


## Create a key table
Map TSX symbols to Yahoo Symbols

In [58]:
yhooMap =  TsxToYhoo()
moc_key_df = yhooMap.run(moc_df)
moc_key_df

PWF.PR.P
PWF-PP.TO
PWF.PR.T
PWF-PT.TO


,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date,yahoo_symbol
0,AAV,BUY,4356,1.575,2020-04-07,AAV.TO
1,AD,BUY,7220,8.670,2020-04-07,AD.TO
2,ADN,BUY,100,12.660,2020-04-07,ADN.TO
3,AFN,BUY,3046,17.325,2020-04-07,AFN.TO
4,AI,BUY,100,9.070,2020-04-07,AI.TO
...,...,...,...,...,...,...
399,WTE,SELL,2012,13.185,2020-04-07,WTE.TO
400,X,SELL,1400,107.640,2020-04-07,X.TO
401,YGR,SELL,1000,0.488,2020-04-07,YGR.TO
402,YRI,SELL,179254,4.695,2020-04-07,YRI.TO


## Add daily features
(from yahoo)

In [52]:
# Get EOD features
eod_df = etl_moc.get_eod_features.run(moc_key_df.head(6))

INFO:root:Error getting info from yahoo for sym AD.TO


## Add indtraday features

In [57]:
intraday_df = etl_moc.get_1min_ohlc.run(moc_key_df.head(6))

[*********************100%***********************]  6 of 6 completed


## Normalize column names

In [64]:
eod_df.columns

/home/ilivni/miniconda3/lib/python3.7/site-packages/prefect/core/task.py:264: UserWarning: DEPRECATED: the result_handler Task option will be deprecated in 0.11.0, and removed in 0.12.0, in favor of the `result` option instead.
  UserWarning,


Index(['Symbol', 'Imbalance Side', 'Imbalance Size',
       'Imbalance Reference Price', 'moc_date', 'yahoo_symbol', 'Date', 'Open',
       'High', 'Low',
       ...
       'dateShortInterest', 'pegRatio', 'lastCapGain', 'shortPercentOfFloat',
       'sharesShortPriorMonth', 'category', 'fiveYearAverageReturn',
       'regularMarketPrice', 'logo_url', 'trailingPE'],
      dtype='object', length=137)

In [62]:
def mormalize_col_names(col_name):
    normed_clmn_name= ''.join(['_' + c.lower() if c.isupper() else c for c in col_name]).lstrip('_')
    return normed_clmn_name

In [65]:
mormalize_col_names("lastCapGain")

'last_cap_gain'

In [35]:
sym = yf.Ticker("AAV.TO")

In [36]:


eod_df.assign([sym.info])

TypeError: assign() takes 1 positional argument but 2 were given

In [37]:
info_df = pd.DataFrame([sym.info])
eod_df.join(info_df)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,yahoo_symbol,zip,sector,...,earningsQuarterlyGrowth,dateShortInterest,pegRatio,lastCapGain,shortPercentOfFloat,sharesShortPriorMonth,category,fiveYearAverageReturn,regularMarketPrice,logo_url
0,1.64,1.65,1.52,1.58,513300,0,0,AAV.TO,T2P 5E9,Energy,...,None,1585612800,84,None,None,8058230,None,None,1.72,https://logo.clearbit.com/advantageog.com


In [ ]:
moc_key_df

In [ ]:
ohlc_df

In [ ]:
df.stack(level=0)

In [ ]:
 df = yf.download(
        
        ['ABX.TO' 'AC.TO' 'ACB.TO' 'AD.TO' 'ADN.TO'], 
        start=st_dt.strftime('%Y-%m-%d'), 
        end=end_dt.strftime('%Y-%m-%d'),
        interval=interval
 )

In [ ]:
for i in grpd_eod_dfs:
    print(i[1]["yahoo_symbol"])

In [ ]:
data = yf.download(
    "SPY AAPL", 
    start="2020-04-06", 
    end="2017-04-30"
)

In [ ]:
# Make daily moc data
daily_moc_df = moc_key_df.merge(
    ohlc_df,
    how="left",
    left_on=["moc_date", "yahoo_symbol"],
    right_on=["Date", "yahoo_symbol"],
    validate="one_to_one"
)

In [ ]:
daily_moc_df = daily_moc_df.drop("Date", axis=1)
daily_moc_df

In [ ]:
daily_moc_df.columns

In [ ]:
clmn_name = 'Imbalance Side'
tmp_col = clmn_name.split(" ")
tmp_col[0].lower()

In [ ]:
def norm_clm_names(clm_name):
    clm_name_lst = 

In [ ]:
daily_moc_df.columns  = map(str.split(" ") ,daily_moc_df.columns) 

In [ ]:
som

In [ ]:
ohlc_df_lst = dailyData.add_ohlc(moc_key_df)

In [ ]:
df_lst = [df for df in ohlc_df_lst]

In [ ]:
ohlc_df = pd.concat(df_lst, axis=0).reset_index()

In [ ]:
ohlc_df.head()

In [ ]:
st_dt = moc_key_df.iloc[0]["moc_date"].strftime('%Y-%m-%d')
st_dt = "2020-04-02"

In [ ]:
sym = yf.Ticker("AEM.TO")

In [ ]:
sym.info["sector"]

In [ ]:
sym.history(
    start="2020-04-02",
    end="2020-04-03",
    interval = "1m",
    auto_adjust = True
)
    

In [ ]:
sym.ticker

In [ ]:
def get_ohlc(row):
    sym = yf.Ticker(row["yahoo_symbol"])
    print(sym.info)
    st_dt = row["moc_date"]
    end_dt = st_dt + timedelta(days=1)
    
    df = sym.history(
            start=st_dt.strftime('%Y-%m-%d'), 
            end=end_dt.strftime('%Y-%m-%d'), 
            auto_adjust=True
        ).head(1)
    
    # Add symbolto ohlc
    df["yahoo_symbol"] = row["yahoo_symbol"]
    print(sym.info)
    #df["sector"] = sym.info["sector"]
    return df

ohlc_df_lst = moc_key_df.head(2).apply(get_ohlc, axis=1)

In [ ]:
ohlc_df_lst[0].rea

In [ ]:
st_dt = moc_key_df.iloc[0]["moc_date"].strftime('%Y-%m-%d')
end_dt = moc_key_df.iloc[0]["moc_date"] # + timedelta(days=1)
st_dt

In [ ]:
sym_to_get_lst[:2]

In [ ]:
data

In [ ]:
return sym.history(start=st_dt).head(1)

In [ ]:
def get_eod_data(row):
    sym = yf.Ticker(row["yahoo_tsx_symbol"])
    row["currency"] = sym.info["currency"]
    
appiled_df = df[""].apply(get_eod_data, axis=1)
df = pd.concat([moc_key_df, appiled_df], axis='columns')

In [ ]:
sym =  moc_key_df.iloc[0]["Symbol"]
sym

In [ ]:
pre_client = Client()
project_nm = "MOC"

In [ ]:
try:
    proj_id = pre_client.create_project(project_name=project_nm)
except Exception as err:
    print(err.with_traceback)

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")